In [58]:
from bs4 import BeautifulSoup 
import requests

# Steps
Using the search word find the URLs for all pages
for each URL
    See if the Page Falls in which category(of the Three).
    
    If First Case
    Then from the URL get all the Pages(max UPTO 10) URL where Reviews are Present
    and Get the Reviews.
    If Second Case then
    Then get Those Reviews then and There.
    If Third Case then 
    Then can't get any reviews.

In [59]:
searchString='iphone'
url = 'https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'.format(searchString)

In [60]:
data=requests.get(url)

In [61]:
Text=BeautifulSoup(data.content,'html.parser')

In [62]:
values=Text.find_all('a',{'class':'_1fQZEK'})
product_url=[]
if len(values)!=0:
    for half_url in values:
        product_url.append('https://www.flipkart.com'+half_url['href'])
else:
    divInfo=Text.find_all('div',{'class':'_2pi5LC col-12-12'})
    for inidi_Div in divInfo:
        temp=inidi_Div.find_all('a',{'class':'_2rpwqI'})
        if len(temp)!=0:
            for temps in temp:
                product_url.append('https://www.flipkart.com'+temps['href'])

In [63]:
len(product_url)

24

In [71]:
reviews=[]
counter_reviews=0
for url in product_url:
    html_url=requests.get(url)
    print("Hitting: "+url+'\n')
    data_url=BeautifulSoup(html_url.content,'html.parser')
    data=data_url.find_all('div',{'class':'_2c2kV-'})
    if len(data)==1:
        """
        No Reviews Posted for that URL Product
        """
        print("No Reviews Posted for that URL Product")
        continue
    elif len(data)==2:
        """
        Reviews Less Than 4 So No 'See All Reviews'.
        """
        print("Reviews Less Than 4 So No 'See All Reviews'")
        temp=data[1].find_all('div',{'class':'_16PBlm'})
        for review in temp:
            temp_dict=dict()
            if review.findChild('p') is None:
                continue
            temp_dict['Heading']=review.findChild('p').get_text()
            temp_dict['Name']=review.findChild('p',{'class':'_2sc7ZR _2V5EHH'}).get_text()
            temp_dict['Rating']=review.findChild('p').find_previous_sibling().get_text()
            temp_dict['Body']=review.find('div',{'class':'t-ZTKy'}).findChild().findChild().get_text()
            if temp_dict not in reviews:
                counter_reviews+=1
                reviews.append(temp_dict)
    else:
        """
        First Get the 'See All Reviews' Link
        Second get all the Reviews from that page and subsequent pages till there are any reviews Left.
        """
        counter=0
        try:
            urls=prev_url='https://www.flipkart.com'+data[-1].find_next_sibling('a')['href']
        except Exception:
            print(data)
            print('Some Unknown Exceptions Occurred.....')
        while counter<5:
            prev_url=urls
            extractData=requests.get(urls)
            print("Hitting: "+urls+'\n')
            extractData=BeautifulSoup(extractData.content,'html.parser')
            try:
                temp=extractData.find('div',{'class':'_1YokD2 _3Mn1Gg col-9-12'}).find_all('div',{'class':'_1AtVbE col-12-12'})
            except Exception:
                print('Halted Due to Exception Occured')
                counter_reviews+=counter
                break
            for review in temp:
                temp_dict=dict()
                if review.findChild('p') is None:
            #_2sc7ZR _2V5EHH
                    continue
                temp_dict['Heading']=review.findChild('p').get_text()
                temp_dict['Name']=review.findChild('p',{'class':'_2sc7ZR _2V5EHH'}).get_text()
                temp_dict['Rating']=review.findChild('p').find_previous_sibling().get_text()
                temp_dict['Body']=review.find('div',{'class':'t-ZTKy'}).findChild().findChild().get_text()
                if temp_dict not in reviews:
                    reviews.append(temp_dict)
                    counter+=1
                    
            if extractData.find('a',{'class':'_1LKTO3'}) is None:
                print("Halted Due to No NEXT Button")
                counter_reviews+=counter
                break
            someData=extractData.find_all('a',{'class':'_1LKTO3'})
            for tag in someData:
                if tag.findChild().get_text()=='Next':
                    urls='https://www.flipkart.com'+tag['href']
            if urls==prev_url:
                print("Halted Due to URLS Match")
                counter_reviews+=counter
                break

Hitting: https://www.flipkart.com/apple-iphone-xr-white-64-gb-includes-earpods-power-adapter/p/itmf9z7zhgzkmgm3?pid=MOBF9Z7ZUYC2EYQD&lid=LSTMOBF9Z7ZUYC2EYQDVYJTIC&marketplace=FLIPKART&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.MOBF9Z7ZUYC2EYQD.SEARCH&ssid=qv0dxfh8r40000001606238593097&qH=0b3f45b266a97d70

Hitting: https://www.flipkart.com/apple-iphone-xr-white-64-gb-includes-earpods-power-adapter/product-reviews/itmf9z7zhgzkmgm3?pid=MOBF9Z7ZUYC2EYQD&lid=LSTMOBF9Z7ZUYC2EYQDVYJTIC&marketplace=FLIPKART

Hitting: https://www.flipkart.com/apple-iphone-se-white-64-gb-includes-earpods-power-adapter/p/itmaf546e27c4eaf?pid=MOBFRFXHMEZRCXRV&lid=LSTMOBFRFXHMEZRCXRVDHSVH1&marketplace=FLIPKART&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.MOBFRFXHMEZRCXRV.SEARCH&ssid=qv0dxfh8r40000001606238593097&qH=0b3f45b266a97d70

Hitting: https://www.flipkart.com/apple-iphone-se-white-64-gb-includes-earpods-po

Hitting: https://www.flipkart.com/apple-iphone-xr-product-red-128-gb-includes-earpods-power-adapter/p/itmf9z7zbkvjzegf?pid=MOBF9Z7ZHQC23PWQ&lid=LSTMOBF9Z7ZHQC23PWQUPEBQY&marketplace=FLIPKART&srno=s_1_10&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.MOBF9Z7ZHQC23PWQ.SEARCH&ssid=qv0dxfh8r40000001606238593097&qH=0b3f45b266a97d70

Hitting: https://www.flipkart.com/apple-iphone-xr-product-red-128-gb-includes-earpods-power-adapter/product-reviews/itmf9z7zbkvjzegf?pid=MOBF9Z7ZHQC23PWQ&lid=LSTMOBF9Z7ZHQC23PWQUPEBQY&marketplace=FLIPKART

Hitting: https://www.flipkart.com/apple-iphone-xr-product-red-128-gb-includes-earpods-power-adapter/product-reviews/itmf9z7zbkvjzegf?pid=MOBF9Z7ZHQC23PWQ&lid=LSTMOBF9Z7ZHQC23PWQUPEBQY&marketplace=FLIPKART&page=2

Hitting: https://www.flipkart.com/apple-iphone-xr-product-red-128-gb-includes-earpods-power-adapter/product-reviews/itmf9z7zbkvjzegf?pid=MOBF9Z7ZHQC23PWQ&lid=LSTMOBF9Z7ZHQC23PWQUPEBQY&marketplace=FLIPKART&page=3



Hitting: https://www.flipkart.com/apple-iphone-12-blue-128-gb/product-reviews/itm02853ae92e90a?pid=MOBFWBYZKPTZF9VG&lid=LSTMOBFWBYZKPTZF9VG6GMIFT&marketplace=FLIPKART

Hitting: https://www.flipkart.com/apple-iphone-12-pro-max-graphite-256-gb/p/itm8dbdf0b986725?pid=MOBFWBYZEF6XQ5ZW&lid=LSTMOBFWBYZEF6XQ5ZWBZ3ZKD&marketplace=FLIPKART&srno=s_1_16&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.MOBFWBYZEF6XQ5ZW.SEARCH&ssid=qv0dxfh8r40000001606238593097&qH=0b3f45b266a97d70

Hitting: https://www.flipkart.com/apple-iphone-12-pro-max-graphite-256-gb/product-reviews/itm8dbdf0b986725?pid=MOBFWBYZEF6XQ5ZW&lid=LSTMOBFWBYZEF6XQ5ZWBZ3ZKD&marketplace=FLIPKART

Halted Due to No NEXT Button
Hitting: https://www.flipkart.com/apple-iphone-xr-blue-128-gb-includes-earpods-power-adapter/p/itmf9z7zegyb2ush?pid=MOBF9Z7Z25H88GT9&lid=LSTMOBF9Z7Z25H88GT9CHSPZX&marketplace=FLIPKART&srno=s_1_17&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.

Hitting: https://www.flipkart.com/apple-iphone-12-mini-black-256-gb/p/itmdd8bad2be7933?pid=MOBFWBYZXYYGEUXK&lid=LSTMOBFWBYZXYYGEUXKF1U2UP&marketplace=FLIPKART&srno=s_1_24&otracker=search&otracker1=search&fm=organic&iid=19181068-710d-486e-87c8-5978a59cf0db.MOBFWBYZXYYGEUXK.SEARCH&ssid=qv0dxfh8r40000001606238593097&qH=0b3f45b266a97d70

Hitting: https://www.flipkart.com/apple-iphone-12-mini-black-256-gb/product-reviews/itmdd8bad2be7933?pid=MOBFWBYZXYYGEUXK&lid=LSTMOBFWBYZXYYGEUXKF1U2UP&marketplace=FLIPKART

Halted Due to No NEXT Button


In [72]:
len(reviews),counter,counter_reviews

(226, 8, 16)

In [66]:
ans=requests.get(product_url[0])

In [73]:
reviews

[{'Heading': 'Must buy!',
  'Rating': '5',
  'Body': 'Really amazing phone by apple .Its come with IOS 13.1.3 it’s have really good features like memoji animoji .😎Simply awesome smart phone👌 I love it😍Thanks apple .Thanks Flipkart.Thanks supercom.'},
 {'Heading': 'Highly recommended',
  'Rating': '5',
  'Body': 'I have bought the 128GB Product RED version. Being an Android user, it was a different hands-on experience of the XR and Iam really liking it.Display looks great, sound quality is great, camera quality is good and mainly the gaming performance is so smooth and fast and never felt a single frame drop ever.Even though its costly for an "Budget" apple iphone, it\'s worth the money I have spent on this mobile.'},
 {'Heading': 'Best in the market!',
  'Rating': '5',
  'Body': 'Best ever from Iphone this is my first apple device and i’m very glad that i bought this instead of OnePlues 7proThe phone has awesome battery backup, sound quality,style , and camera'},
 {'Heading': 'Fabulous

In [68]:
Exception?